# 自然言語処理にむけて

最後の仕上げとして、自然言語処理を学びます。

今回、使うモジュールの準備

In [2]:
import spacy

## 形態素解析

言語における意味の基本単位は語 (word) です。まず、語を取り出す方法からみていきましょう。

### 英語と日本語

自然言語処理は、言語の種類によって難しさや扱い方が異なります。

* (英語文) I bought a book 
* (日本語文) 私は本を買った

英語は、空白で区切られたものを語と考えることができます。
したがって、Python の標準文字列ライブラリだけで、簡単に語を取り出すことができます。

__英語の字句解析__

In [1]:
s = "I bought a book"
s.split()

['I', 'bought', 'a', 'book']

日本語では、まず語の区切りを判定する必要があります。
しかし、この語の区切りを判定するのが かなりの難処理です。

この難処理を行ってくれるのが形態素解析ライブラリです。






### spaCy/GINZA

spaCy は、Explosion AI 社の開発するオープンソースの自然言語処理ライブラリです。2019 年に、 リクルート AI 研究所と国立言語研究所の研究成果である GiNZA が登場し、実用的な日本語処理が 手軽に利用できるようになりました。

まずは、GiNZA の導入から始めましょう。`pip install ginza` を入力するだけで、 spaCy を含めて、自然言語処理に必要なライブラリがまとめてインストールされます。


In [6]:
import spacy
nlp = spacy.load('ja_ginza') 

doc = nlp("私は本を買った") #形態素解析
for word in doc:
    print(word.i, word.orth_, word.lemma_, word.pos_, word.tag_)

0 私 私 PRON 代名詞
1 は は ADP 助詞-係助詞
2 本 本 NOUN 名詞-普通名詞-一般
3 を を ADP 助詞-格助詞
4 買っ 買う VERB 動詞-一般
5 た た AUX 助動詞


<div class="alert alert-info">

Let's try

自然言語処理と形態素解析の良い練習問題は、「自然言語処理１００本ノック」にあります。

http://www.cl.ecei.tohoku.ac.jp/nlp100/

Web上には、解説記事がたくさん掲載されていますので、参考にしながら解いてみると実力がつきます。

</div>


## 演習問題

<div class="alert alert-info">

Let's try

`2 ** (1//2)` が、正しく $\sqrt{2}$ にならない理由を考えてみよう

</div>

<div class="alert alert-warning">

(プログラミングの)$N$個数える

プログラミングでは、原則、「**0から**$N-1$まで」のように数えます。

</div>

### 平均点

<div class="admonition tip">

**例題（平均点）**

期末試験は5人受験した。
点数が40点未満の生徒は全員，補習を受け，成績が40点になった。
5人の平均点を求めよ。

入力例：
```
10
65
100
30
95
```

出力例：
```
68
```

[AtCoder (JOI2014 予選)](https://atcoder.jp/contests/joi2014yo/tasks/joi2014yo_a)

</div>

__(解法) リストを使う場合__

1. 期末試験を記録する空の得点リスト `scores` を用意する
2. 5人分繰り返し、点数を読んで、`scores` に追加する 
3. 平均点は `sum(scores) // 5`
